In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import pickle




from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.impute import KNNImputer



In [2]:
df = pd.read_csv("Sri Lankan CKD Dataset.csv")

In [3]:
a= {"M":1,"F":0}

df["gender"]=df["gender"].map(a)

In [4]:
df = df.drop("cru",axis =1)
df = df.drop("ua",axis =1)
df = df.drop("iron",axis =1)

In [5]:
missing_values_per_row = df.isnull().sum(axis=1)

print("Number of Missing Values per Row:")
print(missing_values_per_row)

Number of Missing Values per Row:
0      6
1      2
2      1
3      2
4      2
      ..
401    4
402    0
403    1
404    4
405    1
Length: 406, dtype: int64


In [6]:
threshold = 10
filtered_data = df[missing_values_per_row <= threshold]

print("\nFiltered DataFrame:")
print(filtered_data)
filtered_data.shape

df = filtered_data.copy()



Filtered DataFrame:
      id   age  weight  gender       cr    al    gl     na    ca    k     cl  \
0      1  72.0   52.00       1   249.43  34.9   NaN  142.0  2.43  5.0  113.0   
1      2  76.0   52.00       1   216.05  41.3   NaN  145.0  2.40  4.7  114.0   
2      3  60.0   80.40       1   155.40  33.7   NaN  135.0  2.00  4.2  103.0   
3      4  65.0   53.40       0   127.50  41.8   NaN  138.0  2.80  4.4  100.0   
4      5  59.0   60.40       1   172.71  39.7   NaN  128.0  2.20  3.6   91.0   
..   ...   ...     ...     ...      ...   ...   ...    ...   ...  ...    ...   
401  402  48.0   48.00       1  1167.62   NaN   NaN  134.0   NaN  4.1   94.0   
402  403  46.0   55.00       1   211.20  37.2  37.9  144.0  2.40  4.3  105.0   
403  404  55.0   54.85       0   164.43  22.2  48.9  129.0   NaN  3.9   97.0   
404  405  84.0   43.90       1   501.71   NaN   NaN  135.0   NaN  5.7   99.0   
405  406  38.0   60.50       1   880.64  38.5  32.1  127.0   NaN  4.1   99.0   

       pr   wbc   

In [7]:
# df = df[['cr','age','hg','al','gender','na','gl','ca','class']].copy()
# df = df[['age','hg','al','gender','na','gl','ca','class']].copy()

df = df[['cr','age','hg','gender','na','class']].copy()





In [8]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=14)

# Perform the imputation
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
df_imputed.head()

,cr,age,hg,gender,na,class
0,249.43,72.0,12.4,1.0,142.0,1.0
1,216.05,76.0,11.3,1.0,145.0,1.0
2,155.40,60.0,8.4,1.0,135.0,1.0
3,127.50,65.0,11.7,0.0,138.0,1.0
4,172.71,59.0,13.8,1.0,128.0,1.0


In [9]:
# df_imputed = df_imputed.drop(columns='id')

In [10]:
# df_imputed = df_imputed.round(1)
X = df_imputed.drop(columns='class')
y = df_imputed['class']

In [11]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# from sklearn.inspection import permutation_importance


In [12]:
# from sklearn.feature_selection import RFE

# model = LogisticRegression(max_iter=1000)

# # Use RFE to select top 2 features
# rfe = RFE(model, n_features_to_select=2)
# fit = rfe.fit(X, y)

# # Print selected features
# selected_features = [feature for feature, selected in zip(X.columns, fit.support_) if selected]
# print(f"Selected Features: {selected_features}")

# # Rank all features
# ranking = pd.DataFrame({'Feature': X.columns, 'Ranking': fit.ranking_})
# print(ranking.sort_values(by='Ranking'))

In [13]:
# # d = df_imputed.round(1)
# d = df_imputed[['ca','gender','hg','al','na','class','cr']]

# X = d.drop(columns='class')
# y = d['class']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train the XGBoost Classifier
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)

# Step 4: Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")



Model Accuracy: 0.96


In [75]:
# X_train

In [76]:
# X_test.round(2)

In [77]:
# y_test

In [78]:
# x_ = np.array([716.230,60.0, 84.0,38.800000,	1.0,121.0,31.042857,2.100000,77.000000,4.5])
# x_ = x_.reshape(1, -1)

# # single_instance = np.array([4.5, 2.3, 1.3, 0.2, 3.1])

# # # Reshape it into a 2D array
# # single_instance_reshaped = single_instance.reshape(1, -1)

In [79]:
# import pickle

# # Load the saved XGBoost model
# with open("model_folder/xgboost_ckd_model.pkl", "rb") as file:
#     m = pickle.load(file)

In [80]:
# y_ = m.predict(x_)
# y_

In [81]:
# y_

In [82]:
y_test

78     0.0
371    1.0
248    0.0
55     1.0
390    1.0
      ... 
364    1.0
82     0.0
114    1.0
3      1.0
18     1.0
Name: class, Length: 79, dtype: float64

In [83]:
import os
folder_name = "model_folder"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created.")

# Step 6: Save the model in the new folder
model_path = os.path.join(folder_name, 'xgboost_ckd_model_no_alglca.pkl')
# model_path = os.path.join(folder_name, 'xgboost_ckd_model.pkl')
# model_path = os.path.join(folder_name, 'xgboost_ckd_model_no_scr.pkl')


with open(model_path, 'wb') as file:
    pickle.dump(model, file)

print(f"Model saved as {model_path}")

Model saved as model_folder\xgboost_ckd_model_no_alglca.pkl
